In [43]:
import torch

In [5]:
# import importlib.metadata
# import tiktoken

# print("tiktoken version:", importlib.metadata.version("tiktoken"))

import importlib.metadata
import tiktoken

print ("tiktoken version", importlib.metadata.version("tiktoken"))

tiktoken version 0.7.0


In [6]:
# Dataset and DataLoader
from torch.utils.data import Dataset, DataLoader
from typing import Any

class GPTDatasetV1(Dataset): # type: ignore
  def __init__(self, txt: str, tokenizer: Any, max_length: int, stride: int):
    self.input_ids: list[int] = []
    self.target_ids: list[int] = []

    token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
    assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i + max_length]
      target_chunk = token_ids[i + 1: i + max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))  # type: ignore
      self.target_ids.append(torch.tensor(target_chunk))# type: ignore

  def __len__(self):
    return len(self.input_ids)
  
  def __getitem__(self, idx: int):
    return self.input_ids[idx],self.target_ids[idx]
  

def create_dataloader_v1(txt: str, batch_size: int = 4, 
                         max_length: int = 256, stride: int=128,
                        shuffle: bool=True, drop_last: bool = True,
                        num_workers: int=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,  # type: ignore
                          drop_last=drop_last, num_workers=num_workers)
  return dataloader

In [19]:
class GPTDatasetV2(Dataset):
  def __init__(self, text: str, tokenizer: Any, max_length: int, stride: int):
    self.input_ids: list[int] = []
    self.target_ids: list[int] = []

    token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
    assert len(token_ids) > max_length, "Number of tokenized inputs must be greater than max_length"

    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i: i+max_length]
      target_chunk = token_ids[i+1: i+max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self): 
    return len(self.input_ids)

  def __getitem__(self, idx: int):
    return self.input_ids[idx], self.target_ids[idx]

def create_dataloader_v2(txt: str, batch_size: int = 4, max_length: int = 256,
                        stride: int = 128, shuffle: bool = True, drop_last: bool = True,
                        num_workers: int = 0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV2(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                          drop_last=drop_last, num_workers=num_workers)
  return dataloader
  

In [18]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

In [20]:
dataloader = create_dataloader_v2(
  raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [21]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [14]:
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  40,  367, 2885, 1464],
        [1807, 3619,  402,  271]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899]])


In [15]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[10899,  2138,   257,  7026],
        [15632,   438,  2016,   257]]), tensor([[ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922]])]


In [22]:
# Creating Token Embeddings

input_ids = torch.tensor([2, 3, 5, 1])


In [24]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [25]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [26]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [27]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [28]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [29]:
max_length = 4
dataloader = create_dataloader_v1(
  raw_text, batch_size=8, max_length=max_length,
  stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [30]:
print("Token IDs:\n", inputs)
print("\nInput shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Input shape:
 torch.Size([8, 4])


In [36]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)


torch.Size([8, 4, 256])


In [38]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 0.3544, -1.1020,  1.6459,  ..., -0.6570, -0.4390, -0.0461],
        [-2.1400, -2.5262,  1.4213,  ..., -1.1446,  2.2331, -1.3071],
        [ 0.2795, -1.1833, -0.0892,  ...,  0.4960,  0.8412,  0.0323],
        [-0.4270, -2.8876, -1.3638,  ..., -0.4342, -0.1795,  1.0636]],
       requires_grad=True)


In [40]:

pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 0.3544, -1.1020,  1.6459,  ..., -0.6570, -0.4390, -0.0461],
        [-2.1400, -2.5262,  1.4213,  ..., -1.1446,  2.2331, -1.3071],
        [ 0.2795, -1.1833, -0.0892,  ...,  0.4960,  0.8412,  0.0323],
        [-0.4270, -2.8876, -1.3638,  ..., -0.4342, -0.1795,  1.0636]],
       grad_fn=<EmbeddingBackward0>)


In [42]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 8.4575e-01,  2.1894e-02,  3.1048e+00,  ..., -1.0565e+00,
          -2.3125e+00, -1.9061e-01],
         [-1.6919e+00, -2.2725e+00,  1.1558e+00,  ..., -6.4488e-01,
           1.0340e+00, -2.4915e+00],
         [ 2.8798e-02, -1.2379e+00,  5.7950e-01,  ...,  1.4578e+00,
           3.2149e+00, -2.0498e-02],
         [ 5.1875e-01, -2.0218e+00,  2.5532e-01,  ..., -8.8868e-01,
          -9.2549e-01,  1.4119e+00]],

        [[ 1.9005e+00,  6.3488e-01,  8.6119e-01,  ..., -7.5740e-01,
           4.1942e-01, -3.8816e-01],
         [-4.0022e+00, -2.7175e+00,  1.0401e+00,  ..., -2.2610e-02,
           1.8835e+00, -6.9800e-01],
         [ 2.2642e+00, -1.8316e+00, -2.3071e-01,  ...,  1.1194e-01,
          -9.4324e-02,  1.4801e+00],
         [ 5.3777e-01, -1.5902e+00, -2.9845e+00,  ...,  7.1201e-01,
           1.4002e+00,  1.4605e+00]],

        [[-4.1687e-01, -4.4478e-01,  1.8122e+00,  ..., -1.4613e+00,
          -3.8482e-01,  6.9653e-01],
         [-1.3354e+00, -2.0